# Notebook Description
This notebook computes the XYZ spatial dependences for both the S1 and S2 yields, in the case of alpha particles. 

First, the Z dependence is computed using fit functions. The Z dependence is modelled as an exponential in the case of S2, and as a 5th degree polynomial for S1. 

Second, once the Z dependence has been corrected for on average, the XY dependence is extracted for both S1 and S2. 

The spatial corrections are returned as best-fit function parameters for Z, and as 2D maps for XY. No smoothing/regularization is done for XY. 

Fiducial alpha events are used to compute these corrections, hence both the inclusive alpha selection and the fiducial selection are applied here.

# Notebook Configuration

In [ ]:
# General configuration
runNumber = 4517
numPMTs = 11

# Configuration for inclusive alpha selection
minNumAllS1s = 1
maxNumAllS1s = 1

minNumAllS2s = 1
maxNumAllS2s = 5

minNumSelectedS1s = 1
maxNumSelectedS1s = 1

minNumSelectedS2s = 1
maxNumSelectedS2s = 1

minS1Charge =  60 # PEs/PMT
maxS1Charge =  1.e6 # PEs/PMT

minS2Charge =  200 # PEs/PMT
maxS2Charge =  1.e6 # PEs/PMT

minS1Width = 0.15 # us
maxS1Width = 1 # us

minS2Width =  3 # us
maxS2Width = 25 # us

minS1StartTime =   0 # us
maxS1StartTime = 649 # us

minS2StartTime = 649 # us
maxS2StartTime = 655 # us

# Configuration for fiducial selection
minZ =       20 # mm
maxZ =      520 # mm
maxRadius = 178 # mm

# Python Imports and Plotting Options

In [ ]:
import os
import functools
import time
from collections import namedtuple


import numpy             as np
import tables            as tb
import pandas            as pd
import matplotlib.pyplot as plt

import invisible_cities.core.core_functions as coref
import invisible_cities.core.fit_functions  as fitf
import invisible_cities.reco.dst_functions  as dstf
import invisible_cities.io  .dst_io         as dstio
import invisible_cities.reco.corrections    as corrf

# import invisible_cities.icaro.hst_functions as hst
# Import JAH's improved hst functions for now, to allow for statistics boxes in histograms.
# Idea is that something like this will be imported in invisible_cities in the future.
import myhst_functions as hst

# Import alpha selection functions
import AlphaSelectionFunctions as asf

print("Running on:", time.asctime())

In [ ]:
plt.style.available
#plt.style.use('seaborn-white')
plt.style.use('seaborn-talk')

optStat = True

# IPython Magic Commands 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# KDST Data Reading and Basic Conversions

In [ ]:
inputFolder   = "/Users/sorel/next/NEXTNEW/data/analysis/{0}/hdf5/dst/".format(runNumber)
inputFilename = inputFolder+"kdst_{0}_icdev_20171017_3_alpha.h5".format(runNumber)


data = dstf.load_dst(inputFilename, "DST", "Events")
data.S1e = data.S1e/numPMTs # convert to PEs/PMT
data.S1w = data.S1w/1.e3 # convert to microseconds
data.S1t = data.S1t/1.e3 # convert to microseconds
data.S1t = data.S1t - data.S1w/2 # convert to peak start time, assuming symmetric peak

data.S2e = data.S2e/numPMTs # convert to PEs/PMT
data.S2w = data.S2w # already microseconds
data.S2t = data.S2t/1.e3 # convert to microseconds
data.S2t = data.S2t - data.S2w/2 # convert to peak start time, assuming symmetric peak

# Print KDST data format
print('KDST columns:',data.columns.values)

In [ ]:
# Print KDST entries and events statistics 
events = data.event
print('Number of KDST entries:', len(events))
print('Number of events: ',      len(events.unique()))

# Inclusive Alpha Selection

In [ ]:
# Inclusive alpha selection
# Goal: keep events with one and only one alpha-like S1, and one and only one alpha-like S2
selInclusiveAlphas, inputEntries, outputEntries = asf.inclusiveAlphaFilter(
    data, 
    minS1Charge,    maxS1Charge,
    minS1Width,     maxS1Width,
    minS1StartTime, maxS1StartTime,
    minS2Charge,    maxS2Charge,
    minS2Width,     maxS2Width,
    minS2StartTime, maxS2StartTime)

print('Inclusive alpha selection:', outputEntries, '/', inputEntries, 'entries')

# From now on, work with data frame reduced according to inclusive alpha selection
data = data[selInclusiveAlphas]
numInclusiveAlphas = sum(selInclusiveAlphas)

In [ ]:
print(numInclusiveAlphas

# Fiducial Selection

In [ ]:
# Fiducial selection
# Selection of KDST entries based on Z and radial position
selFiducial, inputEntries, outputEntries = asf.fiducialFilter(
    data,
    minZ, maxZ,
    maxRadius)

print('Fiducial selection:', outputEntries, '/', inputEntries, 'entries')